### Importação das Bibliotecas

In [1]:
import os
from os.path import dirname
import pandas as pd
from pathlib import Path
import numpy as np
from pypdf import PdfReader
import seaborn as sns

import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from unidecode import unidecode

import plotly.express as px
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer

import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')


[nltk_data] Downloading package punkt to /home/kunumi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kunumi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/kunumi/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

### Definição de Parâmetros

In [2]:
pd.set_option('display.max_colwidth', None)

directory = Path(os.path.dirname(os.getcwd()))

dir_file_input = (directory / "data/02_intermediate/df_beyonce_concat_raw.xlsx")
dir_file_export = (directory / "data/02_intermediate/df_beyonce_concat_cleaned.xlsx")

### Importação de dados

In [3]:
df = pd.read_excel(dir_file_input)
df

,Text
0,Beyoncé
1,Beyoncé em 2023
2,Nome
3,completoBeyoncé Giselle
4,Knowles
...,...
2230,"utilizar o nosso website, estará a concordar com todos os cookies de"
2231,acordo com nossa Política de Cookies. Ler mais
2232,MOSTRAR DETALHES
2233,×Biograﬁa de Beyoncé - eBiograﬁa https://www.ebiograﬁa.com/beyonce/


### Limpeza dos dados

#### Definição de Stopwords

In [4]:
stop_words = set(stopwords.words('portuguese'))

#inclusão de gírias
girias = ["tô", "lá", "né", "bro", "miga", "top", "falou", "vei",
          "tá", "aí", "vai", "hoje", "então", "assim", "porque", 
          "aqui", "acho", "gente", "vou", "sei", "ali",
          "bah", "mano", "minas", "parça", "treta", "zuera", 
          "firmeza", "dahora", "pô", "show", "trampo", "bagulho", 
          "pika", "vacilo", "mó", "massa", "demorou", "rox", 
          "mandar", "firme", "sarado", "catioro", "bicho"]

stop_words.update(girias)

stop_words_personalizadas = ["então", "gente", "live", "kunumi",
                             "bom dia", "boa tarde", "boa noite", "hein",
                             "aline rossin"]

stop_words.update(stop_words_personalizadas)

#Inclusão de todas as palavras do set porém sem acento
new_set = {unidecode(item) for item in stop_words}
stop_words.update(new_set)

#### Limpeza dos Dados

In [5]:
def text_preprocessing_ptbr(text, _stop_words):
    
    # Converter para letras minúsculas e remover acentos
    text = unidecode(str(text).lower())
    
    # Remover números
    text = re.sub(r'\d+', '', text)
    
    # Remover pontuação e caracteres especiais
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenização
    tokens = word_tokenize(text, language='portuguese')
    
    # Remover stopwords e caracteres únicos
    filtered_tokens = [word for word in tokens if len(word) > 1 and word not in _stop_words]
    
    # Reunir tokens em um texto limpo
    cleaned_text = ' '.join(filtered_tokens)

    return cleaned_text

In [6]:
df_cleaned = pd.DataFrame(columns=["Text"])
df_cleaned["Text"] = df["Text"].apply(text_preprocessing_ptbr, _stop_words=stop_words)

df_cleaned.drop_duplicates(inplace=True)
df_cleaned.dropna(inplace=True)

In [9]:
df_cleaned.sample(10)

,Text
1514,forbes consultado maio beyonce wikipedia enciclopedia livre httpsptwikipediaorgwikibeyoncca
144,lancado grupo abriu alguns shows tlc
156,demissao somente ver videoclipe say my name processo arquivado resolvido
1277,consultado agosto ligacao inativa
1977,excelentes trabalhos atriz primeira experiencia noremover anuncios
1579,uol consultado maio beyonce surpreende lancar nova musica durante pandemia ouca blackparade httpsffolhauolcombrmusicabeyoncesurpreendeaolancarnovamusicadurantepandemiaoucablackparadeshtml junho
1476,fevereiro beyonce sasha fierce is done httpwwwrapupcombeyoncesashafierceisdone rap up consultado marco divas pop gravam videoclip favela rio httpsarchiveiszgui dn
788,arrogancia intensidade mostrando poderosa voz perder nenhuma nota muitas
1059,sobre cosmeticos homens mulheres todos produtos utilizados centro forambeyonce wikipedia enciclopedia livre httpsptwikipediaorgwikibeyoncca
1042,gravacoes filme cadillac records onde interpreta cantora


In [10]:

lista_exclusao_linhas = ["html", "http", "2023"]

def exclude_rows_with_words(df, word_list):
    condition = df["Text"].apply(lambda text: not any(word in text for word in word_list))
    return df[condition]

df_cleaned = exclude_rows_with_words(df_cleaned, lista_exclusao_linhas)
df_cleaned.reset_index(inplace=True, drop=True)

#### Exportação dos Dados

In [11]:
df_cleaned.to_excel(dir_file_export)

### Plots

In [12]:
# Função para obter as n palavras mais comuns e suas frequências
def get_most_common_words(df, n=30):
    words_list = ' '.join(df["Text"]).split()
    word_counter = Counter(words_list)
    most_common_words = word_counter.most_common(n)
    most_common_words.sort(key = lambda x: x[1])
    return most_common_words

In [13]:
def get_most_rare_words(df, n=30):
    words_list = ' '.join(df["Text"]).split()
    word_counter = Counter(words_list)
    most_common_words = word_counter.most_common()
    most_rare_words = most_common_words[-n:]  # Pegar as últimas n palavras (mais raras)
    most_rare_words.sort(key = lambda x: x[1])
    return most_rare_words

In [14]:

# Função para obter as n palavras mais comuns e suas frequências usando TF-IDF
def get_most_relevant_words(df, n=30):
    # Cria o vetorizador TF-IDF
    vectorizer = TfidfVectorizer(max_features=5000)  # Use max_features para limitar o número de palavras consideradas
    
    # Aplique o vetorizador ao texto para obter as matrizes TF-IDF
    tfidf_matrix = vectorizer.fit_transform(df['Text'])

    # Obtenha o vocabulário (palavras) a partir do vetorizador
    words = vectorizer.get_feature_names_out()

    # Calcula a soma das frequências TF-IDF de cada palavra
    tfidf_sum_per_word = tfidf_matrix.sum(axis=0).A1

    # Cria um dicionário com as palavras e suas frequências TF-IDF
    words_tfidf = dict(zip(words, tfidf_sum_per_word))

    # Obtenha as palavras mais comuns com base nas frequências TF-IDF
    most_common_words_tfidf = Counter(words_tfidf).most_common(n)
    
    most_common_words_tfidf.sort(key = lambda x: x[1])

    return most_common_words_tfidf

In [15]:
def plot_words(list_words, n=30, topic="Comuns"):
    
    # Obter as palavras e suas frequências separadamente
    words, frequencies = zip(*list_words)

    # Criar o DataFrame para o gráfico
    df_plot = pd.DataFrame({'Palavras': words, 'Frequência': frequencies})

    # Plotar o gráfico de barras horizontais com escala de cores
    fig = px.bar(df_plot, x='Frequência', y='Palavras', orientation='h',
                 color='Frequência', color_continuous_scale='solar',
                 labels={'Frequência': 'Frequência', 
                         'Palavras': 'Palavras Mais {}'.format(topic)},
                 text='Frequência',
                 title="As {} Palavras Mais {}".format(n, topic))
    
     # Configurar o layout do gráfico
    fig.update_layout(
            height=800,
            width=1400 
    )
    # Exibir o gráfico
    fig.show()

#### Palavras mais comuns

In [16]:
most_common_words = get_most_common_words(df_cleaned, n=30)
plot_words(most_common_words, 30, "Comuns")

#### Palavras mais Raras

In [17]:
most_rare_words = get_most_rare_words(df_cleaned, n=30)
plot_words(most_rare_words, 30, "Raras")

#### Palavras mais Relevantes

In [18]:
most_relevant_words_tfidf = get_most_relevant_words(df_cleaned, n=30)
plot_words(most_relevant_words_tfidf, 30, "Relevantes")

In [19]:
all_words = [word.lower() for post in df_cleaned['Text'] for word in word_tokenize(post)]
word_freq = Counter(all_words)
mean_freq = sum(word_freq.values()) / len(word_freq)


def count_rare_words(post):
    post_words = [word.lower() for word in word_tokenize(post)]
    rare_words_count = sum(1 for word in post_words if word_freq[word] < mean_freq)
    return rare_words_count


df_cleaned['rare_words_count'] = df_cleaned['Text'].apply(count_rare_words)
df_cleaned

/tmp/ipykernel_356835/1645028447.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Text,rare_words_count
0,beyonce,0
1,nome,0
2,completobeyonce giselle,1
3,knowles,0
4,outros nomes queen,1
...,...,...
1485,data nascimento,0
1486,biografias lidas,2
1487,monteiro lobatokarl marxromero brittotarsila amaralnelson mandelageorge washingtonthomas edisonthomas jeffersonbenjamin franklinjohn wesleyartemisbiografias atualizadas,12
1488,betinho bhagavan sri sathya sai baba menuc graus todos direitos reservados,10


In [20]:
text_full = ' '.join(df_cleaned["Text"])

tokens = word_tokenize(text_full)
    
stopwords = nltk.corpus.stopwords.words('portuguese')
new_stopwords = ['por','A', 'O', 'pra', 'sobre', 'voce', 'todos', 'dia', 'ja', 'sao', 'hoje', 
             'nao', 'obrigado', 'toda', 'todo', 'agora']
stopwords.extend(new_stopwords)
filtered_tokens = [token for token in tokens if token not in stopwords and token not in string.punctuation]
    
word_freq = nltk.FreqDist(filtered_tokens)
    
unique_word_freq = sum(1 for word in word_freq if word_freq[word] == 1)

In [21]:
word_freq

FreqDist({'beyonce': 384, 'album': 141, 'musica': 100, 'the': 75, 'ano': 65, 'filme': 63, 'primeiro': 62, 'grupo': 58, 'milhoes': 55, 'turne': 49, ...})